In [2]:
from tira.rest_api_client import Client
import pandas as pd
from wows_eval import evaluate
from autoqrels.zeroshot import GradedMonoPrompt

pd.set_option('display.max_colwidth', None)

DATASET_ID = 'wows-eval/pointwise-smoke-test-20250128-training'

tira = Client()

In [4]:
input_data = tira.pd.inputs(DATASET_ID)
input_data.head(3)

,id,query,unknown
0,32d23068-7440-4891-9958-42325f98a604,who sings monk theme song,This is a reference to the minor controversy that brewed among Monk fans over the introduction of the new theme song It's A Jungle Out There written and performed by Randy Newman in the second season of Monk.
1,cde83146-ac3e-4bc5-a959-f2006ac7b8de,who sings monk theme song,"Walker, Texas Ranger. Chuck Norris thought “Eyes of a Ranger” would be the perfect theme song for his new show Walker, Texas Ranger. He wanted his friend Randy Travis should sing it, but CBS had a different idea: The network suggested Norris sing the theme himself."
2,cb7b20d0-def6-46c4-ae44-a78f00b47735,who sings monk theme song,"However, as Brave 's soundtrack reveals, the movie is also noteworthy for being one of the studio's most musical films, especially for one not featuring music by Pixar's go-to songwriter Randy Newman."


In [5]:
BACKBONE_MODEL = "flan-t5-small"

PROMPT = """Instruction: Indicate if the passage answers the question.
###
Example 1:
Question: At about what age do adults normally begin to lose bone mass?
Passage: For most people, bone mass peaks during the third decade of life. By this age, men typically have accumulated more bone mass than women. After this point, the amount of bone in the skeleton typically begins to decline slowly as removal of old bone exceeds formation of new bone.
Answer: Perfectly relevant
###
Example 2:
Question: when and where did the battle of manassas take place
Passage: Summary of the Battle of Bull Run. The conflict took place close to Manassas Junction, Virginia. Around 35,000 Union soldiers marched from Washing D.C. towards Bull Run (a small river) where a 20,000 troop Confederate force was stationed.
Answer: Irrelevant
###
###
Example 3:
Question: what is lbm in body composition
Passage: They also measured the participants body fat and clean body mass of muscle mass, obtained by subtracting the body fat weight from the total body weight.
Answer: Relevant
###
Example 4:
Question: {{ query_text }}
Passage: {{ unk_doc_text }}
Answer:"""

In [6]:
flan_t5_small_assessor = GradedMonoPrompt(
    backbone=f'google/{BACKBONE_MODEL}',
    prompt=PROMPT,
    dataset=None
)

In [7]:
predictions = flan_t5_small_assessor.predict(input_data)

predictions.head(3)

100%|██████████| 2/2 [00:09<00:00,  4.72s/it]


,id,query,unknown,probability_relevant
0,32d23068-7440-4891-9958-42325f98a604,who sings monk theme song,This is a reference to the minor controversy that brewed among Monk fans over the introduction of the new theme song It's A Jungle Out There written and performed by Randy Newman in the second season of Monk.,0.263609
1,cde83146-ac3e-4bc5-a959-f2006ac7b8de,who sings monk theme song,"Walker, Texas Ranger. Chuck Norris thought “Eyes of a Ranger” would be the perfect theme song for his new show Walker, Texas Ranger. He wanted his friend Randy Travis should sing it, but CBS had a different idea: The network suggested Norris sing the theme himself.",0.240609
2,cb7b20d0-def6-46c4-ae44-a78f00b47735,who sings monk theme song,"However, as Brave 's soundtrack reveals, the movie is also noteworthy for being one of the studio's most musical films, especially for one not featuring music by Pixar's go-to songwriter Randy Newman.",0.280311


In [9]:
evaluate(
    predictions,
    DATASET_ID,
    upload=True,
    system_name=f'auto-qrels-pointwise-{BACKBONE_MODEL}',
    system_description="We use autoqrels [1] with a custom in-context learning prompt for pointwise relevance judgments.\n\n[1] - https://github.com/seanmacavaney/autoqrels"
)

The dataset "task_1/foo-pointwise-20250130_0-training" is not publicly available in TIRA. Please visit https://tira.io/datasets for an overview of all public datasets.


ValueError: The dataset "task_1/foo-pointwise-20250130_0-training" is not publicly available in TIRA. Please visit https://tira.io/datasets for an overview of all public datasets.